In [19]:
%reload_ext jetisu.query_idr_magic

# Australian Goods and Services Tax (GST) Example

The following is an example intensionally defined relation that captures
the relationship between consumer Price, the Goods and Services Tax
(GSTAmount), and the price before application of the GST (ExGSTAmount).
The definition is written in MiniZinc.

In [20]:
%%jetisu_show
australian_gst

```

predicate australian_gst(var float: price,
                         var float: ex_gst_amount,
                         var float: gst_amount) =
let {
    constraint price/11 = gst_amount;
    constraint ex_gst_amount = price-gst_amount;
    }
in true;
```

In [21]:
%%jetisu_query
select * from australian_gst where price = 110;

|price|ex_gst_amount|gst_amount|
|----|----|----|
|110.0|100.0|10.0|

The above definition may be read as follows:

  - `predicate` is the MiniZinc term meaning "relation".

  - The intensionally defined relation is given the name
    `Australian_GST`.

  - Declaration of headings (attributes) using `var <type>: <name>`.

  - Two declarations of the relationships between attributes using
    constraints.

The constraints above, should be read in light of the fact that the
following constraints all express the same relationship between Price
and GSTAmount: 

```

constraint Price/11 = GSTAmount;
constraint GSTAmount = Price/11;
constraint GSTAmount*11 = Price;
constraint Price = GSTAmount*11;
```

The following is the behaviour of this intensionally defined relation
under various queries. The `Australian_GST` relation returns an empty
extension when insufficiently constrained to lead to a finite extension.

    select * from Australian_GST;

| price | ExGSTAmount | GSTAmount |
| :---- | :---------- | :-------- |

Query result is an empty relation.

The same empty result is obtained when the relation is constrained in a
manner that violates the intension. For example:


In [22]:
%%jetisu_query
select * from Australian_GST where price=100 and ex_gst_amount=1 and gst_amount=1;

|price|ex_gst_amount|gst_amount|
|----|----|----|


When, however, any one of its attributes is constrained to a value, it
returns a single tuple extension.

In [23]:
%%jetisu_query
select * from australian_gst where gst_amount=10 ;

|price|ex_gst_amount|gst_amount|
|----|----|----|
|110|100|10.0|

And the same result is obtained with any of the following queries.

```
select * from Australian_GST where ExGSTAmount = 100;
select * from Australian_GST where GSTAmount = 10;
select * from Australian_GST where GSTAmount = 10 and Price=110;
select * from Australian_GST where GSTAmount = 10 and ExGSTAmount=100;
select * from Australian_GST where ExGSTAmount = 100 and Price=110;
select * from Australian_GST where ExGSTAmount = 100 and Price=110 and GSTAmount=10;
```
e.g.

In [24]:
%%jetisu_query
select * from Australian_GST where ex_gst_amount = 100;

|price|ex_gst_amount|gst_amount|
|----|----|----|
|110|100.0|10|

The above example demonstrates how programmers do not need to repeat
themselves "DRY”  when driving computations in different directions.
This is in sharp contrast to procedural and functional programming,
whereby the GST computation needs to be restated multiple times,
depending on what value is known and which ones are unknown.
